# Bone Fracture Image Transformations  
Interactive **Negative** & **Gamma** correction demo (with image upload).


In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, FileUpload, VBox, Output, HTML
from IPython.display import display
import os, random


In [ ]:
def plot_image_and_hist(image, title):
    """Draw image + histogram with normalized counts and visible bars."""
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
  
    ax[0].imshow(image, cmap='gray')
    ax[0].set_title(title)
    ax[0].axis('off')
    
    hist, bins = np.histogram(image.ravel(), bins=256, range=(0, 256))

    hist_norm = hist / hist.max()
  
    ax[1].bar(bins[:-1], hist_norm, width=1, color='gray')
    ax[1].set_title("Histogram (normalized)")
    ax[1].set_xlim([0, 256])
    ax[1].set_ylim([0, 1.05]) 
    ax[1].set_ylabel("Normalized Count")
    
    plt.tight_layout()
    plt.show()


In [ ]:
folder = r"C:\Users\user\Downloads\bone fracture detection.v4-v4.yolov8\train\images"
valid_ext = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")

candidates = [f for f in os.listdir(folder) if f.lower().endswith(valid_ext)]
if not candidates:
    raise FileNotFoundError("No valid images in folder")

rand_path = os.path.join(folder, random.choice(candidates))
print("Random default:", rand_path)

original = cv2.imread(rand_path, cv2.IMREAD_GRAYSCALE)
if original is None:
    raise FileNotFoundError("Could not read default image")

negative = 255 - original


Random default: C:\Users\user\Downloads\bone fracture detection.v4-v4.yolov8\train\images\image2_2819_png.rf.18a2eccfcc7211e5724d7162d4bef51c.jpg


In [ ]:
gamma_slider = FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='Gamma', continuous_update=False
)

upload = FileUpload(accept='image/*', multiple=False)
out = Output()

def show_gamma(g):
    out.clear_output(wait=True)
    gamma_corrected = np.array(255 * (original / 255) ** g, dtype='uint8')
    with out:
        plot_image_and_hist(original, "Original")
        plot_image_and_hist(negative, "Negative")
        plot_image_and_hist(gamma_corrected, f"Gamma {g:.1f}")

def handle_upload(change):
    global original, negative, rand_path
    if not upload.value:
        return

    if isinstance(upload.value, tuple):
        up_file = upload.value[0]
        file_bytes = up_file.content
        filename = up_file.name
    else:
        up_file = next(iter(upload.value.values()))
        file_bytes = up_file['content']
        filename = up_file['metadata']['name']

    original = load_image_gray(file_bytes)
    negative = 255 - original
    rand_path = "Uploaded:" + filename 
    print(f"Loaded uploaded image: {filename}")
    show_gamma(gamma_slider.value)

upload.observe(handle_upload, names='value')
gamma_slider.observe(lambda c: show_gamma(gamma_slider.value), names='value')


In [ ]:
title_html = HTML(
    "<h2 style='color:#2c3e50; font-family:sans-serif;'>"
    "Interactive Bone X-ray Transformations</h2>"
)

display(VBox([title_html, upload, gamma_slider, out]))

show_gamma(gamma_slider.value)
